# 서울시 쓰레기 종량제 봉투 최대/최소 비용 구하기

### 모듈 호출

In [120]:
import json
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen, Request
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from pandas import json_normalize
import requests

### REST API 호출

In [121]:
url = 'http://api.data.go.kr/openapi/tn_pubr_public_weighted_envlp_api'

queryParams = '?' + urlencode(
    {quote_plus('ServiceKey'): 'APIKEY',
     quote_plus('pageNo'): '1', quote_plus('numOfRows'): '50', quote_plus('type'): 'json', quote_plus('CTPRVN_NM') : '서울특별시'
     })

request = Request(url + queryParams)

request.get_method = lambda: 'GET'

response_body = urlopen(request, timeout=60).read()

print(response_body)

b'{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE","type":"json"},"body":{"items":[{"ctprvnNm":"\xec\x84\x9c\xec\x9a\xb8\xed\x8a\xb9\xeb\xb3\x84\xec\x8b\x9c","signguNm":"\xea\xb4\x80\xec\x95\x85\xea\xb5\xac","weightedEnvlpType":"\xea\xb7\x9c\xea\xb2\xa9\xeb\xb4\x89\xed\x88\xac","weightedEnvlpMthd":"\xec\x86\x8c\xea\xb0\x81\xec\x9a\xa9","weightedEnvlpPrpos":"\xec\x83\x9d\xed\x99\x9c\xec\x93\xb0\xeb\xa0\x88\xea\xb8\xb0","weightedEnvlpTrget":"\xea\xb0\x80\xec\xa0\x95\xec\x9a\xa9","price1":"0","price1Half":"0","price2":"0","price2Half":"0","price3":"90","price5":"130","price10":"250","price20":"490","price30":"740","price50":"1250","price60":"0","price75":"1880","price100":"0","price120":"0","price125":"0","chrgDeptNm":"\xec\xb2\xad\xec\x86\x8c\xed\x96\x89\xec\xa0\x95\xea\xb3\xbc","phoneNumber":"02-879-6206","referenceDate":"2021-10-15","insttCode":"3200000"},{"ctprvnNm":"\xec\x84\x9c\xec\x9a\xb8\xed\x8a\xb9\xeb\xb3\x84\xec\x8b\x9c","signguNm":"\xea\xb4\x80\xec\x95\x85

### API 디코딩 및 JSON 파일 저장

In [122]:
tempDic = json.loads(response_body)

data = json.dumps(tempDic, indent=4, ensure_ascii=False)
print(data)

with open('./TrashPackPrice.json', 'w', encoding='utf-8') as file:
    file.write(data)

{
    "response": {
        "header": {
            "resultCode": "00",
            "resultMsg": "NORMAL_SERVICE",
            "type": "json"
        },
        "body": {
            "items": [
                {
                    "ctprvnNm": "서울특별시",
                    "signguNm": "관악구",
                    "weightedEnvlpType": "규격봉투",
                    "weightedEnvlpMthd": "소각용",
                    "weightedEnvlpPrpos": "생활쓰레기",
                    "weightedEnvlpTrget": "가정용",
                    "price1": "0",
                    "price1Half": "0",
                    "price2": "0",
                    "price2Half": "0",
                    "price3": "90",
                    "price5": "130",
                    "price10": "250",
                    "price20": "490",
                    "price30": "740",
                    "price50": "1250",
                    "price60": "0",
                    "price75": "1880",
                    "price100": "0",
                    "pric

### 데이터 전처리

In [123]:
with open('./TrashPackPrice.json', 'r', encoding='utf-8') as file:
    json_sample = json.load(file)

# 불필요한 정보 제거
json_data = pd.json_normalize(json_sample['response']['body']['items'])
json_data = json_data.drop(['chrgDeptNm', 'phoneNumber', 'referenceDate', 'insttCode', 'weightedEnvlpMthd', 'ctprvnNm'], axis=1)
json_data = json_data.drop(['price125', 'price120', 'price100'], axis=1)
json_data = json_data.drop(['price1', 'price2', 'price3', 'price5', 'price1Half', 'price2Half', 'price75', 'price60', 'price50', 'price30', 'price10'], axis=1)

# 사업장용 봉투 제거
temp_index = json_data[json_data['weightedEnvlpTrget'] == '사업장용'].index
json_data = json_data.drop(temp_index)

# 영업용 봉투 제거
temp_index = json_data[json_data['weightedEnvlpTrget'] == '영업용'].index
json_data = json_data.drop(temp_index)

# 용도 불문 기타 봉투 제거
temp_index = json_data[json_data['weightedEnvlpTrget'] == '기타'].index
json_data = json_data.drop(temp_index)

# 20리터 종량제 봉투를 제외한 나머지 봉투 제거
temp_index = json_data[json_data['price20'] == '0'].index
json_data = json_data.drop(temp_index)

# 음식물 쓰레기 봉투 제거
temp_index = json_data[json_data['weightedEnvlpPrpos'] == '음식물쓰레기'].index
json_data = json_data.drop(temp_index)

json_data

,signguNm,weightedEnvlpType,weightedEnvlpPrpos,weightedEnvlpTrget,price20
0,관악구,규격봉투,생활쓰레기,가정용,490
2,서초구,규격봉투,생활쓰레기,가정용,490
4,송파구,재사용규격봉투,생활쓰레기,가정용,490
5,송파구,재사용규격봉투,생활쓰레기,가정용,490
8,은평구,규격봉투,생활쓰레기,가정용,490
9,은평구,규격봉투,생활쓰레기,가정용,760
10,은평구,재사용규격봉투,생활쓰레기,가정용,490
11,은평구,특수규격마대,생활쓰레기,가정용,2040
13,중구,특수규격마대,생활쓰레기,가정용,2040
19,서초구,재사용규격봉투,생활쓰레기,가정용,490


### 퀵정렬 함수 선언

In [124]:
def partition(list, start, end):
    pivot = list[start]
    left = start + 1
    right = end
    done = False
    while not done:
        while left <= right and list[left] <= pivot:
            left += 1
        while left <= right and list[right] > pivot:
            right -= 1
        if right < left:
            done = True
        else:
            list[left], list[right] = list[right], list[left]
    list[start], list[right] = list[right], list[start]  # 피봇 교환
    return right

def QuickSort(list, start, end):
    stack = []
    stack.append(start)
    stack.append(end)

    while stack:
        end = stack.pop()
        start = stack.pop()
        pivot = partition(list, start, end)

        if pivot - 1 > start:
            stack.append(start)
            stack.append(pivot - 1)

        if pivot + 1 < end:
            stack.append(pivot + 1)
            stack.append(end)

    return list

### 데이터프레임값 리스트로 변환 및 정렬

In [125]:
price_list = list(np.array(json_data['price20'].tolist()))
price_list = list(map(int, price_list))
price_list = QuickSort(price_list, 0, len(price_list)-1)
price_list = list(map(str, price_list))

### 쓰레기 봉투 최소 값

In [126]:
minPrice = price_list[0]

search_index = json_data[json_data['price20'] == minPrice].index
json_data.loc[search_index]

,signguNm,weightedEnvlpType,weightedEnvlpPrpos,weightedEnvlpTrget,price20
0,관악구,규격봉투,생활쓰레기,가정용,490
2,서초구,규격봉투,생활쓰레기,가정용,490
4,송파구,재사용규격봉투,생활쓰레기,가정용,490
5,송파구,재사용규격봉투,생활쓰레기,가정용,490
8,은평구,규격봉투,생활쓰레기,가정용,490
10,은평구,재사용규격봉투,생활쓰레기,가정용,490
19,서초구,재사용규격봉투,생활쓰레기,가정용,490
21,중구,규격봉투,생활쓰레기,가정용,490
22,중구,재사용규격봉투,생활쓰레기,가정용,490
23,성북구,규격봉투,생활쓰레기,가정용,490


### 쓰레기 봉투 최대 값

In [127]:
maxPrice = price_list[len(price_list)-1]

search_index = json_data[json_data['price20'] == maxPrice].index
json_data.loc[search_index]

,signguNm,weightedEnvlpType,weightedEnvlpPrpos,weightedEnvlpTrget,price20
11,은평구,특수규격마대,생활쓰레기,가정용,2040
13,중구,특수규격마대,생활쓰레기,가정용,2040
29,구로구,특수규격마대,생활쓰레기,가정용,2040
36,성동구,특수규격마대,생활쓰레기,가정용,2040
37,성동구,특수규격마대,생활쓰레기,가정용,2040
44,노원구,특수규격마대,생활쓰레기,가정용+사업장용,2040
49,동대문구,특수규격마대,생활쓰레기,가정용,2040
